In [1]:
import pandas as pd
from scipy.io import mmread
import scanpy as sc
import os
import sys
sys.path.append(os.path.abspath('../../src'))
import SortedNoDupeBedOverlap as bdO
import io_utils
import stats_utils

In [2]:
 mat, id2bc, id2peak = io_utils.read_atac_count_10x('./filtered_peak_bc_matrix')

In [3]:
## check number of cells and peaks
print('number of peaks: %d' % len(id2peak))
print('number of cells: %d' % len(id2bc))

number of peaks: 74633
number of cells: 36286


In [4]:
## read cell type specific peaks downloaded from GSE147672
ref_subtype_peaks0, ref_bk_peak = io_utils.read_reference_data('../../data/GSE97887/', verbose = False)

In [5]:
intersect = set(bdO.BedOverlap(id2peak, sorted(ref_bk_peak), .5))

ref_subtype_peaks = []
for c, peaks in ref_subtype_peaks0:
    ref_subtype_peaks.append( (c, set(bdO.BedOverlap(id2peak, sorted(peaks), .5))) )

In [6]:
## number of cell type specific peaks 
for c, peaks in ref_subtype_peaks:
    print(c, len(peaks))

Astrocytes-1 9040
Astrocytes-2 7109
Hippocampal excitatory-1 12002
Hippocampal excitatory-2 16021
Isocortical Astrocytes 18961
Isocortical excitatory 25980
Isocortical inhibitory 17669
Microglia 11076
Nigral Astrocytes 17073
Nigral OPCs 7092
Nigral cells 6269
Nigral neurons 8498
OPCs-1 10331
OPCs-2 16968
Olidendrocytes-1 17555
Olidendrocytes-2 21231
Olidendrocytes-3 21920
Olidendrocytes-4 21165
Olidendrocytes-5 11400
Striatal Astrocytes 12571
Striatal inhibitory 9440


In [ ]:
## calculate enrichment scores for every cell
scores = stats_utils.compute_enrichment_score(mat, intersect, id2peak, ref_subtype_peaks, num_cores = 10)

In [1]:
## visualization using scanpy
pcas = stats_utils.normalized_pca_from_sparse_mat(mat, n_pc = 20) 
            
data_obj = sc.AnnData(pcas, 
      obs = pd.DataFrame([], index = id2bc),
      )        

sc.pp.neighbors(data_obj, use_rep='X')
sc.tl.umap(data_obj) 

data_obj.obs = score_df

sc.pl.umap(data_obj, color = score_df.columns, cmap = 'RdBu_r', vmax = 5, show = False)

NameError: name 'stats_utils' is not defined

In [ ]:
score_df = pd.DataFrame(scores, 
                        columns = [c for c, _ in ref_subtype_peaks],
                       index = data_obj.obs.index
                       )

In [ ]:
score_df.head()

In [ ]:
data_obj.obs = score_df

In [ ]:
## plotting odds ratio on umap, red colors shows cells that are enriched in cell-type specific peaks
sc.pl.umap(data_obj, color = score_df.columns, cmap = 'RdBu_r', vmax = 5)